# Importing the datasets 


In [27]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer

true = pd.read_csv('../raw_data/True.csv', nrows = 1000)
fake = pd.read_csv('../raw_data/Fake.csv', nrows = 1000)

In [28]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [29]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


# Dropping unseful features - Move to a binary classification problem

In [30]:
true.drop(columns = ['date', 'subject'], inplace = True)
fake.drop(columns = ['date', 'subject'], inplace = True)

In [31]:
true['score']=1
fake['score']=0

# Concatenating the datasets

In [32]:
data = pd.concat([true,fake], ignore_index=True)


In [33]:
stop_words =['/Getty Images']
pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
data['text'] = data['text'].str.replace(pat, '')


# Basic preprocessing

In [34]:
import string
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
data['text'] = data['text'].map(remove_punctuation)
data['title'] = data['title'].map(remove_punctuation)
def lower_case(text):
    text = text.lower()
    return text
data['text'] = data['text'].map(lower_case)
data['title'] = data['title'].map(lower_case)
def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text
data['text'] = data['text'].map(remove_numbers)
data['title'] = data['title'].map(remove_numbers)













In [35]:
data.head()

,title,text,score
0,as us budget fight looms republicans flip thei...,washington reuters the head of a conservative...,1
1,us military to accept transgender recruits on ...,washington reuters transgender people will be...,1
2,senior us republican senator let mr mueller do...,washington reuters the special counsel invest...,1
3,fbi russia probe helped by australian diplomat...,washington reuters trump campaign adviser geo...,1
4,trump wants postal service to charge much more...,seattlewashington reuters president donald tr...,1


In [36]:
x = data['text']
y = data['score']

#y.reshape((2000,1))


# Splitting

In [37]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)

# Grid search for the best model


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
# Create Pipeline
pipeline = Pipeline([
    ('cvt', CountVectorizer()),
    ('nb', MultinomialNB()),
])
# Set parameters to search
parameters = {
    'cvt__ngram_range': ((1,1), (2,2),(3,3),(4,4)),
    'nb__alpha': (0.1,1,2,5,10),}
# Perform grid search
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,
                           scoring = "accuracy",
                           refit=True, cv=5, verbose = 1)
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

# Prediction

In [ ]:
grid_search.predict(x_test)

In [ ]:
x_test

In [ ]:
grid_search.score(x_test, y_test)

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range= (2, 2))
x_train_bow = vectorizer.fit_transform(x_train)


In [40]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 10)
model.fit(x_train_bow,y_train)


#model.score(x_test,y_test)

MultinomialNB(alpha=10, class_prior=None, fit_prior=True)

In [41]:
x_test_bow = vectorizer.transform(x_test)
    
    
model.score(x_test_bow,y_test)   

0.9883333333333333

In [42]:
test = pd.Series(['USA attacked Cina in 2008'])

test =  vectorizer.transform(test)

model.predict(test)

array([0])